In [ ]:
%load_ext autoreload
%autoreload 2
%pdb 0

In [1]:
import pandas as pd
import numpy as np
import csrar
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [ ]:
data = pd.read_csv('../data/Injector_Median100k.csv')

In [2]:
import arff
file = open('test3.arff', 'r')
dataset = arff.load(file)
data = pd.DataFrame(dataset['data'])

#### About test2.arff:
useful=Vector(0, 1)  
dependent=Vector(5, 6, 7, 8, 9)  
duplicated=Vector((0,12), (4,10), (6,11))  
nominal=List()  
clusters=[(0, 1, 2), (3, 4)]  
distribution=List(24860, 25140)  
seed=0
#### About test3.arff
useful=Vector(0, 1)  
dependent=Vector(5, 6, 7, 8, 9)  
duplicated=Vector((0,12), (4,10), (6,11))  
nominal=List()  
clusters=[(0, 1, 2), (3, 4)]  
distribution=List(249786, 250214)  

In [3]:
X = data.drop(13, axis=1)
y = data[13]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [4]:
rar_data = pd.concat([X_train, y_train], axis=1)

In [5]:
rar_data.rename(columns=lambda c: str(c), inplace=True)
# rar_data

In [6]:
target = '13'
rar = csrar.rar.RaR(rar_data)

In [ ]:
test = rar.correlation

In [7]:
rar.run(target, k=5, runs=120)

1. 12
2. 0
3. 7
4. 2
5. 11
6. 6
7. 3
8. 5
9. 1
10. 8
11. 9
12. 4
13. 10


In [ ]:
selected_features = list(map(lambda f: int(f[0]), rar.feature_ranking[:5]))

In [ ]:
selected_features

In [ ]:
X_train_s = X_train[selected_features]
X_test_s = X_test[selected_features]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(X_train['Col132'], linestyle='', marker='.')
plt.show()

In [ ]:
plt.plot(y_train, linestyle='', marker='.')
plt.show()

In [ ]:
clf = ExtraTreesClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn import tree
import pydot
from sklearn.externals.six import StringIO
dot_data = StringIO()
tree.export_graphviz(clf, out_file='tree.dot') 
# graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
# graph[0].write_pdf("tree.pdf") 

In [ ]:
for index, p in enumerate(clf.predict(X_test)):
    print('{}. {}'.format(index+1, p))

In [ ]:
y_test

In [ ]:
clf_selected = ExtraTreesClassifier()
clf_selected.fit(X_train_s, y_train)
clf_selected.score(X_test_s, y_test)

In [ ]:
ranking = []
for index, importance in enumerate(clf.feature_importances_):
    ranking.append((data.columns[index], importance))
ranking.sort(key=lambda r: r[1], reverse=True)
for (index, rank) in enumerate(ranking):
        print('{}. {} with a score of {}'.format(index + 1, rank[0], rank[1]))

In [ ]:
selected_features2 = [r[0] for r in ranking[:5]]
selected_features2

In [ ]:
X_train_s2 = X_train[selected_features2]
X_test_s2 = X_test[selected_features2]

In [ ]:
clf_selected2 = ExtraTreesClassifier()
clf_selected2.fit(X_train_s2, y_train)
clf_selected2.score(X_test_s2, y_test)